In [0]:
import pyspark.sql.functions as sf
from time import time
spark.conf.set("spark.sql.session.timeZone", "America/New_York")

In [0]:
# How mmuch time should be analyzed in seconds
window_time = time() - 43200


In [0]:
# Read data from 'runescape.02_silver.item_mapping'
df_map = spark.read.table("runescape.02_silver.item_mapping")

In [0]:
df_map = df_map.filter("ID IN ('1955','2315','2317')") #Cooking Apple, Pie Shell, Uncooked Apple Pie

In [0]:
df_map.display()

In [0]:
# Read data from 'runescape.02_silver.latest_prices_cleansed'
# and filter to the last 12 hours
df = spark.read.table("runescape.02_silver.latest_prices_enriched").filter(f"time >= {window_time}")
df = df.withColumn("NYCTime", sf.from_unixtime('time'))
df = df.filter("ID IN ('1955','2315','2317')")
df.count()

In [0]:
# Read data from 'runescape.01_bronze.latest_prices_raw'
df_raw = spark.read.table("runescape.01_bronze.latest_prices_raw")

# Read data from 'runescape.01_bronze.latest_prices_raw'
df_raw = df_raw.filter("ID IN ('1955','2315','2317')")
df_raw = df_raw.filter(f"time >= {window_time}") # exclude older data
#df2 = df2.dropDuplicates()
#df2 = df2.sort("NYCTime", ascending=False)
df_raw.display()

Databricks visualization. Run in Databricks to view.

In [0]:
# Read data from 'runescape.01_bronze.latest_prices_raw'
df_raw = spark.read.table("runescape.01_bronze.latest_prices_raw")

# Read data from 'runescape.01_bronze.latest_prices_raw'
df_raw = df_raw.filter("ID IN ('1955')")
df_raw = df_raw.filter(f"time >= {window_time}") # exclude older data
#df2 = df2.dropDuplicates()
#df2 = df2.sort("NYCTime", ascending=False)
df_raw.display()

Databricks visualization. Run in Databricks to view.

In [0]:
# Read data from 'runescape.01_bronze.1h_prices' to get Volumes
# TODO replace this read with the new latest_1_prices table that will have the latest 1 hour price for each item
df_avg = spark.read.table("runescape.01_bronze.1h_prices")

In [0]:
# Get average of 1 hour price data over the last 24 hours
# TODO replace this with actual 24 hour calculation based on underlying raw data
df_24_hours_avg = df_avg.\
    filter(f"time >= {window_time}").\
    filter("ID IN ('1955','2315','2317')").\
    groupBy("id").\
    agg(sf.round(sf.avg("avg1HourHigh"), 2).alias("avg_24hr_High")
    , sf.round(sf.avg("avg1HourHighVolume"), 2).alias("avg_24hr_High_Volume")
    , sf.round(sf.avg("avg1HourLow"), 2).alias("avg_24hr_Low")
    , sf.round(sf.avg("avg1HourLowVolume"), 2).alias("avg_24hr_Low_Volume"))




In [0]:
%skip

# Filter to only use the most recent Volume data
max_time = df_avg.select(sf.max(df_avg.time))

#df_avg = df_avg.filter("time == max_time.first('time'")
df_avg = df_avg.join(max_time, df_avg.time == sf.col("max(time)"))
df_avg = df_avg.drop("max(time)").drop("time")
#df_avg.display()




In [0]:
df2 = df
# filter out low limit items
# df2 = df2.filter(sf.col("limit") >= 70)
# filter to only sell data
# df2 = df2.filter(sf.col("highorlow") == "high")

In [0]:

df3 = df2.withColumn("high_Alch_Margin", sf.round((sf.col("highalch") - sf.col("price")) - 100,1))
# Join with 24 hour data to get volume
df3 = df3.join(df_24_hours_avg, "id",)
#df3 = df3.filter("avg_24hr_High_Volume >= 100")
#df3 = df3.filter("high_Alch_Margin >= 400")
#df3 = df3.sort("high_Alch_Margin", ascending=False)
df3.display()

TODO figure out how to make a chart of the difference from/ profit margin between uncut and cut rubys

In [0]:
df_ruby_cutting = df3.filter("id IN ('1619','1603')")
df_ruby_cutting.display()


Databricks visualization. Run in Databricks to view.